## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim

In [494]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [495]:
params = get_config('../../data/config/delft.json')  # load configuration

#params.times.pickup_patience = 540 # 1 hour of simulation
params.times.patience = 180
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 50 # vehicles


## Parameters for ExMAS

In [496]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2 

#params.platforms.comm_rate = 0.2 
#params.shareability.comm_rate = float(0.30)

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True


# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)

In [497]:
params.kpi = 3

In [498]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

08-08-23 12:37:19-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
08-08-23 12:37:19-INFO-creating combinations
08-08-23 12:37:19-INFO-9900	 nR*(nR-1)
08-08-23 12:37:19-INFO-Reduction of feasible pairs by 99.14%
08-08-23 12:37:19-INFO-Degree 2 	Completed
08-08-23 12:37:19-INFO-trips to extend at degree 2 : 173
08-08-23 12:37:20-INFO-At degree 2 feasible extensions found out of 6 searched
08-08-23 12:37:20-INFO-Degree 3 	Completed
08-08-23 12:37:20-INFO-trips to extend at degree 3 : 6
08-08-23 12:37:20-INFO-At degree 3 feasible extensions found out of 0 searched
08-08-23 12:37:20-INFO-Degree 4 	Completed
08-08-23 12:37:20-INFO-Max degree reached 4
08-08-23 12:37:20-INFO-Trips still possible to extend at degree 4 : 0


In [499]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

08-08-23 12:37:20-INFO-Matching 100 trips to 279 rides in order to min u_veh
08-08-23 12:37:20-INFO-Problem solution: Optimal. 
Total costs for single trips:         27,454 
reduced by matching to:               22,589


In [500]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],8.713500,471,1,[8.7135],"[0, 471]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 244183905, 3216669367]","[None, 0, 0]",node time req_id od 0 ...,471,4.71,7.065000,1.413000,5.6520
1,[1],4.466000,241,1,[4.466],"[27, 241]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44858073, 44825831]","[None, 1, 1]",node time req_id od 0 NaN...,241,2.41,3.622500,0.724500,2.8980
2,[2],4.813000,260,1,[4.813],"[59, 260]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44771638, 44827063]","[None, 2, 2]",node time req_id od 0 NaN...,260,2.60,3.903000,0.780600,3.1224
3,[3],9.157500,495,1,[9.157499999999999],"[65, 495]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1519889905, 1391531777]","[None, 3, 3]",node time req_id od 0 ...,495,4.95,7.425000,1.485000,5.9400
4,[4],8.385000,453,1,[8.385],"[102, 453]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44831009, 1552650568]","[None, 4, 4]",node time req_id od 0 ...,453,4.53,6.799500,1.359900,5.4396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,"[39, 43, 54]",22.445000,895,31,"[11.1094, 7.834250000000001, 3.5013500000000004]","[1380.0, 241, 252, 199, 144, 59]","[39, 43, 54]","[54, 39, 43]",3,274,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 313902614, 3510592637, 1584502763, 1402...","[None, 39, 43, 54, 54, 39, 43]",node time req_id od 0 ...,895,8.95,13.936500,2.787300,11.1492
275,"[0, 4, 7]",19.985750,681,32,"[7.853575, 7.8865, 4.245675]","[-7.5, 117, 119, 272, 40, 133]","[0, 4, 7]","[0, 7, 4]",3,275,...,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[None, 244183905, 44831009, 3669138204, 321666...","[None, 0, 4, 7, 0, 7, 4]",node time req_id od 0 ...,681,6.81,13.012875,2.602575,10.4103
276,"[39, 43, 52]",22.965325,898,31,"[11.2186, 7.75235, 3.994375]","[1380.0, 241, 100, 249, 249, 59]","[39, 43, 52]","[52, 43, 39]",3,276,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 313902614, 3510592637, 44866496, 626009...","[None, 39, 43, 52, 52, 43, 39]",node time req_id od 0 ...,898,8.98,14.506875,2.901375,11.6055
277,"[39, 43, 54]",22.181100,866,31,"[11.245899999999999, 7.4338500000000005, 3.501...","[1380.0, 241, 252, 199, 115, 59]","[39, 43, 54]","[54, 43, 39]",3,277,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 313902614, 3510592637, 1584502763, 1402...","[None, 39, 43, 54, 54, 43, 39]",node time req_id od 0 ...,866,8.66,13.936500,2.787300,11.1492


# calculations 

In [501]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],8.713500,471,1,[8.7135],"[0, 471]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 244183905, 3216669367]","[None, 0, 0]",node time req_id od 0 ...,471,4.71,7.065000,1.413000,5.6520
1,[1],4.466000,241,1,[4.466],"[27, 241]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44858073, 44825831]","[None, 1, 1]",node time req_id od 0 NaN...,241,2.41,3.622500,0.724500,2.8980
2,[2],4.813000,260,1,[4.813],"[59, 260]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44771638, 44827063]","[None, 2, 2]",node time req_id od 0 NaN...,260,2.60,3.903000,0.780600,3.1224
3,[3],9.157500,495,1,[9.157499999999999],"[65, 495]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1519889905, 1391531777]","[None, 3, 3]",node time req_id od 0 ...,495,4.95,7.425000,1.485000,5.9400
4,[4],8.385000,453,1,[8.385],"[102, 453]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44831009, 1552650568]","[None, 4, 4]",node time req_id od 0 ...,453,4.53,6.799500,1.359900,5.4396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,"[39, 43, 54]",22.445000,895,31,"[11.1094, 7.834250000000001, 3.5013500000000004]","[1380.0, 241, 252, 199, 144, 59]","[39, 43, 54]","[54, 39, 43]",3,274,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 313902614, 3510592637, 1584502763, 1402...","[None, 39, 43, 54, 54, 39, 43]",node time req_id od 0 ...,895,8.95,13.936500,2.787300,11.1492
275,"[0, 4, 7]",19.985750,681,32,"[7.853575, 7.8865, 4.245675]","[-7.5, 117, 119, 272, 40, 133]","[0, 4, 7]","[0, 7, 4]",3,275,...,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[None, 244183905, 44831009, 3669138204, 321666...","[None, 0, 4, 7, 0, 7, 4]",node time req_id od 0 ...,681,6.81,13.012875,2.602575,10.4103
276,"[39, 43, 52]",22.965325,898,31,"[11.2186, 7.75235, 3.994375]","[1380.0, 241, 100, 249, 249, 59]","[39, 43, 52]","[52, 43, 39]",3,276,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 313902614, 3510592637, 44866496, 626009...","[None, 39, 43, 52, 52, 43, 39]",node time req_id od 0 ...,898,8.98,14.506875,2.901375,11.6055
277,"[39, 43, 54]",22.181100,866,31,"[11.245899999999999, 7.4338500000000005, 3.501...","[1380.0, 241, 252, 199, 115, 59]","[39, 43, 54]","[54, 43, 39]",3,277,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 313902614, 3510592637, 1584502763, 1402...","[None, 39, 43, 54, 54, 43, 39]",node time req_id od 0 ...,866,8.66,13.936500,2.787300,11.1492


In [502]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

08-08-23 12:37:22-INFO-Matching 100 trips to 279 rides in order to min u_veh
08-08-23 12:37:22-INFO-Problem solution: Optimal. 
Total costs for single trips:         27,454 
reduced by matching to:               22,589
0.0005
08-08-23 12:37:25-CRITICAL-this is request 0 with [0, 173, 174, 175, 176, 262, 263, 264, 265, 275] available rides.
08-08-23 12:37:25-CRITICAL-ride 0 available [1]
08-08-23 12:37:25-CRITICAL-ride 173 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 174 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 175 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 176 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 262 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 263 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 264 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 265 available [1, 0]
08-08-23 12:37:25-CRITICAL-ride 275 available [1, 0, 0]
08-08-23 12:37:25-CRITICAL-this is request 0 with [0, 173, 174, 175, 176, 262, 263, 264, 265, 275] still available rides.
08-08-23

In [503]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,568.0,20.0,30.0,306.0,182.0,0.0,10.0,0,0,0,0,0,0,568.0,488.0,55.0,False
1,15.0,241.0,20.0,30.0,0.0,0.0,27.0,10.0,0,0,0,0,0,0,241.0,0.0,55.0,False
2,15.0,250.0,20.0,30.0,24.0,164.0,59.0,10.0,0,0,0,0,0,0,250.0,188.0,55.0,False
3,15.0,536.0,20.0,30.0,3.0,333.0,65.0,10.0,0,0,0,0,0,0,536.0,336.0,55.0,False
4,15.0,614.0,35.0,30.0,453.0,80.0,87.0,10.0,0,0,0,0,0,0,614.0,533.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.0,363.0,20.0,30.0,7.0,54.0,3359.0,10.0,0,0,0,0,0,0,363.0,61.0,55.0,False
96,15.0,225.0,35.0,30.0,0.0,3210.0,143.0,10.0,0,0,0,0,0,0,225.0,3210.0,55.0,False
97,15.0,203.0,20.0,30.0,65.0,0.0,3397.0,10.0,0,0,0,0,0,0,203.0,65.0,55.0,False


In [504]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,DRIVING_TIME,DRIVING_DIST,FARE,COMMISSION,REVENUE,PICKUP_DIST,TRAVEL_DIST,OPERATING_COST,PROFIT,LOST_PATIENCE
veh,,,,,,,,,,,,,,,,,,,,,
1,3.0,0.0,False,0,0,0.0,15.0,0,237.0,0,...,972.0,9.72,3.6225,0.7245,2.8980,5.07,7.48,0.3740,2.5240,False
2,4.0,0.0,False,0,0,2501.0,30.0,0,250.0,0,...,1088.0,10.88,3.9030,0.7806,3.1224,3.12,5.72,0.2860,2.8364,False
3,4.0,0.0,False,0,0,2313.0,45.0,0,154.0,0,...,1189.0,11.89,7.4250,1.4850,5.9400,4.73,9.68,0.4840,5.4560,False
4,5.0,0.0,False,0,0,2416.0,45.0,0,118.0,0,...,913.0,9.13,6.7995,1.3599,5.4396,3.02,7.55,0.3775,5.0621,False
5,1.0,0.0,False,0,0,2573.0,15.0,0,20.0,0,...,127.0,1.27,5.8665,1.1733,4.6932,0.01,3.92,0.1960,4.4972,False
6,5.0,0.0,False,0,0,2088.0,45.0,0,204.0,0,...,931.0,9.31,3.5370,0.7074,2.8296,2.17,4.52,0.2260,2.6036,False
7,2.0,0.0,False,0,0,2103.0,30.0,0,40.0,0,...,675.0,6.75,3.4860,0.6972,2.7888,1.73,4.05,0.2025,2.5863,False
8,0.0,0.0,False,0,0,357.0,0.0,0,0.0,0,...,0.0,0.00,4.1130,0.8226,3.2904,0.00,2.74,0.1370,3.1534,True
9,4.0,0.0,False,0,0,1710.0,30.0,0,108.0,0,...,1034.0,10.34,3.1815,0.6363,2.5452,3.65,5.77,0.2885,2.2567,False


# Service rate 

In [505]:
Fulfilled_requests = sim.res[0].veh_exp.LOST_PATIENCE.value_counts().loc[False]
unFulfilled_requests = sim.res[0].veh_exp.LOST_PATIENCE.value_counts().loc[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')



service_rate =  80.0 %


In [506]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv')

In [507]:
#for i in range(1, 2):
    #params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    #sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
   # sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
   

In [508]:
#sim.vehs[2].rdf 

In [509]:
#pd.DataFrame(sim.vehs[2].myrides)

## Script 